### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

### Pegando os jogos do dia

In [2]:
# Obter a data de hoje
hoje = date.today()
ontem = hoje - timedelta(days=1)

### Atualizando os datasets que tem jogo hoje

In [3]:
dia = ontem

In [4]:
# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)

Jogos do dia 2023-11-09



100%|██████████| 97/97 [01:25<00:00,  1.14it/s]


In [5]:
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Dataset de temporada atualizado com 9 jogos.

----------------------------------------------------------

Atenção! 27 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
53,2023-11-09,Islândia Premier League,16:15,Breidablik,Njardvik,NaN,NaN,Não iniciado
68,2023-11-09,Paraguai Lnb,20:00,Olimpia,Dep. San Jose,NaN,NaN,Não iniciado
70,2023-11-09,Eua Ncaa,21:00,Kent State,James Madison,NaN,NaN,Não iniciado
71,2023-11-09,Eua Ncaa,21:00,South Florida,South Carolina St,NaN,NaN,Não iniciado
72,2023-11-09,Eua Ncaa,21:00,Florida Intl,Tarleton,NaN,NaN,Não iniciado
73,2023-11-09,Eua Ncaa,21:00,Elon,East Tennessee St,NaN,NaN,Não iniciado
74,2023-11-09,Eua Ncaa,21:00,Charleston Southern,North Florida,NaN,NaN,Não iniciado
76,2023-11-09,Eua Ncaa,21:00,Stonehill,Army,NaN,NaN,Não iniciado
77,2023-11-09,Eua Ncaa,21:00,Tulane,Northwestern St.,NaN,NaN,Não iniciado
78,2023-11-09,Eua Ncaa,21:00,William & Mary,American,NaN,NaN,Não iniciado
